In [2]:
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer

vec = cPickle.load(open('../../preprocess/allfields_with_embedding_19995.p', 'rb'))
cohen_vec = cPickle.load(open('../../preprocess/cohendata_dedup_19995.p'))
#da_vec = cPickle.load(open('../data/vectorizers/decision_aids_vec_5000.p'))

In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv('../data/files/test_cohen_dedup.csv')

nb_studies = len(df)
H = np.zeros((nb_studies, nb_studies))

cdnos = list(set(df.cdno))
for i in range(nb_studies) :
    H[i, df[df['cdno'] == df['cdno'][i]].index] = 1
    
H[np.arange(nb_studies), np.arange(nb_studies)] = 0

In [5]:
index = vec.index['abstract']
vec.X = vec.X[index[0]:index[1]]
train_X = vec.X
train_X = map(lambda s : [vec.idx2word[t] for t in s if t > 1], train_X)

In [6]:
test_X = cohen_vec.X
test_X = map(lambda s : [vec.idx2word[t] for t in s if t > 1], test_X)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=lambda i:i, lowercase=False)
train_TF = tfidf.fit_transform(train_X)

In [8]:
test_TF = tfidf.transform(test_X)
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
from sklearn.preprocessing import normalize
# embedds_n = normalize(np.array(test_TF.todense()), 'l2')
# scores = np.dot(embedds_n, embedds_n.T)
scores = cosine_similarity(test_TF)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000

In [26]:
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print pd.DataFrame(pd.Series(rocs))

0.775436690503
                                          0
ACEInhibitors_processed            0.771045
ADHD_processed                     0.776352
Antihistamines_processed           0.736155
AtypicalAntipsychotics_processed   0.746382
BetaBlockers_processed             0.658662
CalciumChannelBlockers_processed   0.660590
Estrogens_processed                0.850782
NSAIDS_processed                   0.826040
Opiods_processed                   0.814721
OralHypoglycemics_processed        0.829010
ProtonPumpInhibitors_processed     0.816751
SkeletalMuscleRelaxants_processed  0.579367
Statins_processed                  0.757510
Triptans_processed                 0.930586


In [ ]:
from sklearn.decomposition import TruncatedSVD
lsa = TruncatedSVD(n_components=800, algorithm='arpack', tol=0.00001)
train_lsa = lsa.fit_transform(train_TF)

In [ ]:
test_lsa = lsa.transform(test_TF)

In [ ]:
from sklearn.preprocessing import normalize
embedds_n = normalize(np.array(test_lsa.todense()), 'l2')
scores = np.dot(embedds_n, embedds_n.T)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000

In [ ]:
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print rocs

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(tokenizer=lambda i:i, lowercase=False)
train_C = count.fit_transform(train_X)
test_C = count.transform(test_X)

In [17]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=20)

In [18]:
train_lda = lda.fit_transform(train_C)

/home/sarthak/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [19]:
test_lda = lda.transform(test_C)

In [20]:
from sklearn.preprocessing import normalize
embedds_n = normalize(test_lda, 'l2')
scores = np.dot(embedds_n, embedds_n.T)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000
scores = np.array(scores)

In [33]:
#10 LDA
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print pd.Series(rocs)

0.727338114753
ACEInhibitors_processed              0.723784
ADHD_processed                       0.832570
Antihistamines_processed             0.670235
AtypicalAntipsychotics_processed     0.877819
BetaBlockers_processed               0.614932
CalciumChannelBlockers_processed     0.665315
Estrogens_processed                  0.598639
NSAIDS_processed                     0.725580
Opiods_processed                     0.799238
OralHypoglycemics_processed          0.695005
ProtonPumpInhibitors_processed       0.735092
SkeletalMuscleRelaxants_processed    0.741728
Statins_processed                    0.661741
Triptans_processed                   0.826750
dtype: float64


In [10]:
#TFIDF 19K
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print pd.DataFrame(pd.Series(rocs))

0.790149201688
                                          0
ACEInhibitors_processed            0.809715
ADHD_processed                     0.896114
Antihistamines_processed           0.810739
AtypicalAntipsychotics_processed   0.746003
BetaBlockers_processed             0.672498
CalciumChannelBlockers_processed   0.665070
Estrogens_processed                0.874923
NSAIDS_processed                   0.848748
Opiods_processed                   0.812200
OralHypoglycemics_processed        0.786787
ProtonPumpInhibitors_processed     0.809734
SkeletalMuscleRelaxants_processed  0.596118
Statins_processed                  0.788765
Triptans_processed                 0.924097
